In [ ]:
import pandas as pd

In [ ]:
df_cluster = pd.read_csv('Cluster_Tweet_Final.csv')

In [ ]:
df_user_sub_community =pd.read_csv('User_SubCommunity_Map.csv')

In [ ]:
all_acc_camp= pd.read_csv('Acc_Campaign_Map.csv')

In [ ]:
df_cluster[df_cluster.User_ID.isin(df_user_sub_community.loc[df_user_sub_community.Community_ID == 0].User_ID)]

In [ ]:
all_acc_camp

In [ ]:
df_user_sub_community.loc[df_user_sub_community.Community_ID == 0].User_ID

In [ ]:
com0_id = all_acc_camp[all_acc_camp.User_ID.isin(df_user_sub_community.loc[df_user_sub_community.Community_ID == 0].User_ID)].groupby('Campaign_ID').count().reset_index()

In [ ]:
com0_id 

In [ ]:
df_cluster.Cluster_ID = df_cluster.Cluster_ID.astype('int')

In [ ]:
df_cluster['Campaign_ID'] = 'Wild_' + df_cluster['Cluster_ID'].astype('str')

In [ ]:
df_cluster[df_cluster.Campaign_ID.isin(com0_id.Campaign_ID)][['Campaign_ID', 'Tweet_Text']].drop_duplicates('Campaign_ID')

In [ ]:
df_english_tweet = pd.read_csv('Translated_Tweet_Topic.csv')

In [ ]:
df_english_tweet.Cluster_ID = df_english_tweet.Cluster_ID.astype('int')

In [ ]:
df_cluster_eng = df_cluster.loc[df_cluster.Tweet_Lang == 'en'].drop_duplicates('Cluster_ID')[['Cluster_ID', 'Tweet_Text']]

In [ ]:
df_cluster_eng 

In [ ]:
df_english_tweet[~df_english_tweet.Cluster_ID.isin(df_cluster_eng.Cluster_ID)]

In [ ]:
appended_df = pd.concat([df_cluster_eng, df_english_tweet[~df_english_tweet.Cluster_ID.isin(df_cluster_eng.Cluster_ID)]], ignore_index=True)

print(appended_df)

In [ ]:
appended_df.loc[appended_df.Campaign_ID == 'Wild_1716']

In [ ]:
appended_df = appended_df.drop('Unnamed: 0', axis = 1)

In [ ]:
appended_df['Campaign_ID'] = 'Wild_'  +  appended_df['Cluster_ID'].astype('str')

In [ ]:
import re
def remove_twitter_mentions(text):
    return re.sub(r'@\w+', '', text)

def remove_links(text):
    return re.sub(r'http\S+', '', text)


In [ ]:
df_community_english_0 = pd.merge(appended_df, com0_id)

In [ ]:
df_community_english_0['Refined_Text'] = df_community_english_0['Tweet_Text'].apply(remove_twitter_mentions)
df_community_english_0['Refined_Text']=df_community_english_0['Refined_Text'].apply(remove_links)

In [ ]:
docs = df_community_english_0['Refined_Text'].tolist()

In [ ]:
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=5)
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model, language="multilingual", hdbscan_model=hdbscan_model)

In [ ]:
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.get_topic_info().iloc[10:20]

In [ ]:
topic_model.generate_topic_labels()

In [ ]:
topics

In [ ]:
indexes = [index for index, value in enumerate(topics) if value == 0]

In [ ]:
docs[indexes[120]]

In [ ]:
len(docs)

In [ ]:
df_community_english_0.loc[df_community_english_0.Campaign_ID == 'Wild_1716']